<img src='https://pennstatelaw.psu.edu/sites/default/files/styles/content_header_image/public/M%20and%20A%20center%201.jpg?itok=nxlHzngRhttps://pennstatelaw.psu.edu/sites/default/files/styles/content_header_image/public/M%20and%20A%20center%201.jpg?itok=nxlHzngR' height="100%" width="3500">

# Projeto Quant.dev

Olá! Seja bem vindo ao projeto de Quant do FEA.dev!

O que você irá fazer:

  "*Você é um sócio e analista quantitativo do nosso Fundo. Seu objetivo é apresentar sua estratégia de investimento e convencer os outros sócios a colocá-la no portfólio. Você tem boa parte do seu patrimônio pessoal investido nesse Fundo.*"

### Você decidiu implementar a seguinte estratégia:

A sua hipótese é que as ações mais negociadas no mês, ou seja, de maior volume, irão bem no mês seguinte. A sua regra de investimento será comprar, todo final de mês, as 10 ações com o maior volume do indice IBX. Você tem um montante inicial de R$100.000 para investir.

Comece extraindo os dados de preços e volume, tome cuidado que nem todas as fontes te dão essas informações. 

A composição do IBX a cada ano lhe foi dada no arquivo excel. O período de sua estratégia deve contemplar todo o período do mesmo arquivo.

Depois que sua estratégia estiver rodando sem erros, faça uma **análise minuciosa** da estratégia (a maior parte da sua avaliação de desempenho se dará nessa parte). Para isso, use métricas diversas (retorno, volatilidade, drawdown, etc.) plotando e comparando-as. Também faça variações dessa estratégia (comprar as 5 maiores, 10 maiores, 15 maiores, etc) e veja qual é a melhor.

Por fim, deixe mais do que claro se:

1. Está confiante para indicar essa ideia para o portfólio. ou  
2. Não chegou a um resultado satisfatório. ou  
3. Acha que precisa de mais estudos para se sentir confiante.

Obs: Não há problema se algumas ações derem erro na hora da extração, siga em frente. Mas se muitas estiverem dando erro, melhor revisar seu código...

### Materiais de apoio:
1. [Coursera: Introduction to Portfolio Construction and Analysis with Python (link)](https://www.coursera.org/programs/universidade-de-sao-paulo-br-on-coursera-mvxtw?currentTab=MY_COURSES&productId=qeyXSV_uEeiNIQqyU-T6bA&productType=course&showMiniModal=true) 

    - Semanas 1, 2 e 3

2. [Livro: Quantitative Trading - Ernest Chan (link)](https://drive.google.com/drive/folders/1pHySMzZjpHyvVOWn6Ect7CsZXmThzyAO)

    - Cap 1: Inteiro
    - Cap 2: 17-31
    - Cap 3: 36-69

3. [Playlist de Finanças Quantitativas do FEA.dev (link)](https://www.youtube.com/playlist?list=PLiJqHBsOKi2cabSus1hgdSNAHeU0md6_D)

As ações mais negociadas no mês terão melhor performance no mês seguinte. Comprar, todo final de mês, as 10 ações com o maior volume do indice IBX. Montante inicial R$100.000.

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from pandas_datareader import data as pdr
import datetime

yf.pdr_override() 

# Lendo a Composição mensal do IBX
comp_ibx = pd.read_csv('composicao_ibx_mensal.csv')

# Cada linha representa um mês. Preciso identificar
# o ticker, por coluna, no site e depois localizar o preço e volume na data.

# Atribuindo todas as ações à variável "tickers"
tickers = comp_ibx.values[:, 1:].astype(str)

# Criando um array com as ações únicas
ticker_unico = np.unique(tickers)

# Excluindo a ação 'nan' e transformando o array em lista
ticker_unico = list(ticker_unico[:-1])

for j, i in enumerate(ticker_unico):
    i = i + ".SA"
    ticker_unico[j] = i

delete = ["FFTL4.SA", "ALLL3.SA", "ALSC3.SA", "CTIP3.SA", "ELPL3.SA", "CNFB4.SA", "MAGG3.SA", "ARTR3.SA", "TERI3.SA", "BISA3.SA", "SUZB5.SA", "NATU3.SA", "TNLP3.SA", "GETI4.SA", "TCSL4.SA", "TAMM4.SA", "PRML3.SA", "OGXP3.SA", "AEDU3.SA", "UGPA4.SA", "TNLP4.SA", "FIBR3.SA", "VVAR11.SA", "CRUZ3.SA", "TMAR5.SA", "PCAR4.SA", "RUMO3.SA", "VALE5.SA", "VIVO4.SA", "ELPL4.SA", "AMIL3.SA", "MPLU3.SA", "AMBV3.SA"]

for i in delete:
    ticker_unico.remove(i)

startD = '2010-11-01'
endD = '2020-02-01'

start_date = pd.to_datetime(startD)
end_date = pd.to_datetime(endD)
data = pdr.get_data_yahoo(ticker_unico, start=start_date, end=end_date)

# usando resample para agrupar dados diários em dados mensais
n = 1
mon_data = pd.DataFrame(data[['Adj Close', 'Volume']].resample('BM').apply(lambda x: x[-1 - n]))

end_of_months = mon_data.index.tolist()
end_of_months[-1] = data.index[-1]
mon_data.index = end_of_months

mon_data.index = mon_data.index - datetime.timedelta(days=n)


# baixando as inf. do Yahoo! Finance
def download_data(ticker_unico):
    return pdr.get_data_yahoo(ticker_unico, start=start_date, end=end_date)[['Adj Close', 'Volume']]


if __name__ == '__main__':
    data = download_data(ticker_unico)

[*********************100%***********************]  142 of 142 completed
[*********************100%***********************]  142 of 142 completed


In [ ]:
# definindo Index mensal, ao invés de diário.
mon_data.index = pd.to_datetime(mon_data.index, format='%Y-%m-%d').to_period("M")

In [ ]:
mon_data

In [ ]:
returns_asset = mon_data['Adj Close'].pct_change()
volume_df = mon_data['Volume']

In [ ]:
import pandas_datareader.data as web

In [ ]:
# DataFrame com dados do benchmarket brasileiro, para efeitos de comparação no futuro.

# Baixando os dados e especificando index mensal
ibov = web.get_data_yahoo('^BVSP',start = '2010-12-01', end = '2020-02-01', interval='1mo')[['Adj Close', 'Volume']]
ibov.index = pd.to_datetime(ibov.index, format='%Y-%m-%d').to_period("M")

# Obtendo os retornos 
returns_ibov = ibov['Adj Close'].pct_change()
ibov

### Comprar, todo final de mês, as n ações com o maior volume do indice IBX. Você tem um montante inicial de R100.000 para investir.

In [ ]:
# Definindo a quantidade n de ações
n = int(input("Digite o número de ações que irá compor sua carteira: "))

# criando um df vazio
maxvolume_df = volume_df.set_index(volume_df.index)
# Selecionando as n ações de maior volume em volume_df e alocando-as em index + 1 (momento da compra)
maxvolume_df = pd.DataFrame(volume_df.columns.values[np.argsort(-volume_df.values, axis=1)[:, :n]], 
                  index=volume_df.index+1,
                  columns = range(n)).reset_index()
maxvolume_df.set_index('index', inplace = True)

# Excluir a última linha, com data == 2020-01
#maxvolume_df = maxvolume_df.drop(maxvolume_df.tail(1).index)

In [ ]:
maxvolume_df

# Análises

In [ ]:
#TESTE
#returns = pd.DataFrame(index=maxvolume_df.index, columns=maxvolume_df.columns)

#for index, linha in maxvolume_df.iterrows():
#    for index_item, valor_linha in linha.iteritems():
#        returns.loc[index+1, index_item] = returns_asset.loc[index, [valor_linha]][0]
    

In [ ]:
# retornos em t+1
returns = pd.DataFrame(index=maxvolume_df.index, columns=maxvolume_df.columns)

for index, linha in maxvolume_df.iterrows():
    for index_item, valor_linha in linha.iteritems():
        if(str(index) < '2020-01' ):
            returns.loc[index+1, index_item] = returns_asset.loc[index+1, [valor_linha]][0]

returns = returns[:-1]

returns

In [ ]:
# A matriz "R" é o retorno do portfolio, considerando um peso de 10% pra cada uma.
peso = (100/len(returns.columns))/100
R = returns*peso # É o mesmo que fazer returns * 0.10, no caso de n = 10.


In [ ]:
# Definindo uma função que nos retorna o montante e retorno de cada período do portfolio.
def portfolio(montante, retornos):
    Rt = retornos.sum(axis = 1) # Calcula o retorno de cada mês
    montante = montante*(1+Rt).cumprod() # Calcula o montante, de acordo com o retorno do mês.
    return pd.DataFrame({
            'Montante': montante,
            'Retornos' : Rt
            }, index=returns.index)


# Com base num montante de 100(mil) e o retorno R do portfolio, temos:
portfolio(100, R)

In [ ]:
# Enquanto isso, o montante e retornos do IBOV são definidos por:
dados_ibov = pd.DataFrame({'Montante': 100*(1+returns_ibov).cumprod(),
                 "Retornos": returns_ibov,
                 }, index = returns_ibov.index)


dados_ibov

In [ ]:
portfolio(100, R)['Retornos'].plot(title = "Retornos do portfolio no período").bar(x = portfolio(100, R)['Retornos'].index, height = portfolio(100, R)['Retornos'])

In [ ]:
dados_ibov['Retornos'].plot(title = "Retornos do IBOV no período").bar(x = dados_ibov['Retornos'].index, height = dados_ibov['Retornos'])

In [ ]:
# Retorno acumulado do portfolio durante todo o período, em porcentagem.
acumulado = (((portfolio(100, R)['Retornos']+1).prod()-1)*100).round(2)
acumulado

In [ ]:
# Retorno acumulado do IBOV durante todo o período, em porcentagem.
acumulado_ibov = (((dados_ibov['Retornos']+1).prod()-1)*100).round(2)
acumulado_ibov

# Drawdown e outras análises

In [ ]:
# Criar uma função que retorne o Wealth Index, Previous Peak e Drawdown de uma série. 

# Drawdown: Quanto 1000 teria rendido, se investido no período inicial?
def drawdown(return_serie: pd.Series):
    wealth_index = 1000*(1+return_serie).cumprod()
    previous_peak = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peak)/previous_peak
    return pd.DataFrame({
        "Wealth Index": wealth_index,
        "Previous Peak": previous_peak,
        "Drawdown": drawdowns
    })

In [ ]:
#Quando ocorreram os piores resultados do porfólio?
portfolio(100,R).idxmin()

In [ ]:
#Quando ocorreram os piores resultados do IBOV?
dados_ibov.idxmin()

In [ ]:
drawdown(portfolio(100,R)['Retornos'])[['Previous Peak', 'Wealth Index']].plot(title = 'Evolução do Montante na estratégia com os picos do período', figsize = (10,5))
drawdown(dados_ibov['Retornos'])[['Previous Peak', 'Wealth Index']].plot(title = 'Evolução do Montante aplicados no IBOV com os picos do período', figsize = (10,5))

In [ ]:
# Comparação dos Drawdowns do IBOV e do portfolio.
drawdown(dados_ibov['Retornos'])['Drawdown'].plot()
drawdown(portfolio(100,R)['Retornos'])['Drawdown'].plot(title = 'Drawdowns do IBOV (azul) vs. do Portfolio (laranja)', figsize = (10,5))

In [ ]:
# Comparação do Wealth Index do IBOV e do portfolio.
drawdown(dados_ibov['Retornos'])['Wealth Index'].plot()
drawdown(portfolio(100,R)['Retornos'])['Wealth Index'].plot(title = 'Wealth Index do IBOV (azul) vs. do Portfolio (laranja)', figsize = (10,5))

# Resumo


In [ ]:
# Análises do Portfólio
std = portfolio(100, R)['Retornos'].std(axis = 0)
mean = portfolio(100, R)['Retornos'].mean(axis = 0)

# RETORNO SIMPLES ANUALIZADO
annualized = (np.sum(R.mean()) * 12)

# Variância do Portfolio no período
squared_deviations= (portfolio(100,R)['Retornos'] - portfolio(100,R)['Retornos'].mean())**2
variance = squared_deviations.sum()/(portfolio(100,R)['Retornos'].shape[0]-1)

# Volatilidade do portfólio
volatility = np.sqrt(variance)

# Volatilidade Anualizada
anual_volatility = volatility*np.sqrt(12)

# Retornos anualidade
anual_ret = (portfolio(100,R)['Retornos'] +1).prod()**(12/portfolio(100,R)['Retornos'].shape[0])-1

# Sharpe Ratio
sharpe = (anual_ret-0.03)/anual_volatility

# Máximo (mínimo) drawdown
draw_port = drawdown_port = drawdown(portfolio(100,R)['Retornos'])['Drawdown'].min()

In [ ]:
# Análises do IBOV
std_ibov = dados_ibov['Retornos'].std(axis = 0)
mean_ibov = dados_ibov['Retornos'].mean(axis = 0)

# RETORNO SIMPLES ANUALIZADO
ibov_anu = float(np.sum(dados_ibov['Retornos'].mean())*12)

# Variância do Portfolio no período
squared_deviations_ibov = (dados_ibov['Retornos'] - dados_ibov['Retornos'].mean())**2
variance_ibov = squared_deviations_ibov.sum()/(dados_ibov['Retornos'].shape[0]-1)

# Volatilidade do portfólio
volatility_ibov = np.sqrt(variance_ibov)

# Volatilidade Anualizada
anual_volatility_ibov = volatility_ibov*np.sqrt(12)

# Retornos anualidade
anual_ret_ibov = (dados_ibov['Retornos'] +1).prod()**(12/dados_ibov['Retornos'].shape[0])-1

# Sharpe Ratio
sharpe_ibov = (anual_ret_ibov-0.03)/anual_volatility_ibov

# Máximo (mínimo) drawdown 
draw_ibov = drawdown(dados_ibov['Retornos'])['Drawdown'].min()

In [ ]:
pd.DataFrame({"IBOV":(str(acumulado_ibov)+"%", str(round(mean_ibov,2)*100)+"%", str(round(ibov_anu,2)*100)+"%", str(round(volatility_ibov,2)*100)+"%", str(round(anual_volatility_ibov,2)*100)+"%", str(round(sharpe_ibov,2)*100)+"%", str(round(draw_ibov,2)*100)+"%", str(round(variance_ibov, 2)*100)),
             "Porfólio":(str(acumulado)+"%", str(round(mean,2)*100)+"%", str(round(annualized, 2)*100)+"%", str(round(volatility, 2)*100)+"%", str(round(anual_volatility,2)*100)+"%", str(round(sharpe,2)*100)+"%", str(round(draw_port,2)*100)+"%", str(round(variance,2)*100)+"%")}, index = ("Retorno Acumulado", "Média Retornos", "Retorno Anualizado", "Volatilidade", "Vol. Anualizada", "Sharpe Ratio", "Max Drawdown", "Variância"))

In [ ]:
# Não consegui :(
returns.cov()